In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np
import gzip
import pickle
import json
import os
import zipfile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import f_regression

In [ ]:
# Cargar los datos
df_entrenamiento = pd.read_csv('../files/input/train_data.csv/train_default_of_credit_card_clients.csv')
df_prueba = pd.read_csv('../files/input/test_data.csv/test_default_of_credit_card_clients.csv')

# Renombrar la columna "default payment next month" a "default"
df_entrenamiento.rename(columns={'default payment next month': 'default'}, inplace=True)
df_prueba.rename(columns={'default payment next month': 'default'}, inplace=True)

# Remover la columna "ID"
df_entrenamiento.drop(columns=['ID'], inplace=True)
df_prueba.drop(columns=['ID'], inplace=True)

# Eliminar registros con información no disponible
df_entrenamiento.dropna(inplace=True)
df_prueba.dropna(inplace=True)

# Agrupar valores de EDUCATION > 4 en la categoría "others"
df_entrenamiento['EDUCATION'] = df_entrenamiento['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
df_prueba['EDUCATION'] = df_prueba['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

In [ ]:
# Dividir los datos en características (X) y etiquetas (y)
x_entrenamiento = df_entrenamiento.drop(columns=['default'])
y_entrenamiento = df_entrenamiento['default']
x_prueba = df_prueba.drop(columns=['default'])
y_prueba = df_prueba['default']


In [ ]:
# Identificar columnas categóricas y numéricas
columnas_categoricas = ['SEX', 'EDUCATION', 'MARRIAGE']
columnas_numericas = [col for col in x_entrenamiento.columns if col not in columnas_categoricas]

# Preprocesamiento para variables categóricas y numéricas
preprocesador = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), columnas_categoricas),
        ('num', 'passthrough', columnas_numericas)
    ])

# Crear el pipeline
pipeline = Pipeline([
    ('preprocesador', preprocesador),
    ('pca', PCA(n_components=0.95)),
    ('escalador', MinMaxScaler()),
    ('selector', SelectKBest(score_func=f_regression)),
    ('clasificador', MLPClassifier(max_iter=30000))
])

In [ ]:
# Definir los hiperparámetros a optimizar
parametros_grid = {
    'selector__k': range(1, len(x_entrenamiento.columns) + 1),
    'clasificador__hidden_layer_sizes': [(n,) for n in range(1, 11)],
    'clasificador__solver': ['adam'],
    'clasificador__learning_rate_init': [0.01, 0.001, 0.0001],
}

# Realizar la búsqueda de hiperparámetros
busqueda_grid = GridSearchCV(pipeline, parametros_grid, cv=10, scoring='balanced_accuracy', refit=True, verbose=2)
busqueda_grid.fit(x_entrenamiento, y_entrenamiento)

# Mejor modelo encontrado
mejor_modelo = busqueda_grid

In [ ]:
# Guardar el modelo comprimido
with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
    pickle.dump(mejor_modelo, f)

In [ ]:
# Predecir en los conjuntos de entrenamiento y prueba
y_entrenamiento_pred = mejor_modelo.predict(x_entrenamiento)
y_prueba_pred = mejor_modelo.predict(x_prueba)

# Calcular métricas para el conjunto de entrenamiento
metricas_entrenamiento = {
    'type': 'metrics',
    'dataset': 'train',
    'precision': precision_score(y_entrenamiento, y_entrenamiento_pred),
    'balanced_accuracy': balanced_accuracy_score(y_entrenamiento, y_entrenamiento_pred),
    'recall': recall_score(y_entrenamiento, y_entrenamiento_pred),
    'f1_score': f1_score(y_entrenamiento, y_entrenamiento_pred)
}

# Calcular métricas para el conjunto de prueba
metricas_prueba = {
    'type': 'metrics',
    'dataset': 'test',
    'precision': precision_score(y_prueba, y_prueba_pred),
    'balanced_accuracy': balanced_accuracy_score(y_prueba, y_prueba_pred),
    'recall': recall_score(y_prueba, y_prueba_pred),
    'f1_score': f1_score(y_prueba, y_prueba_pred)
}

# Calcular la matriz de confusión para el conjunto de entrenamiento
mc_entrenamiento = confusion_matrix(y_entrenamiento, y_entrenamiento_pred)
mc_entrenamiento_dict = {
    'type': 'cm_matrix',
    'dataset': 'train',
    'true_0': {'predicted_0': int(mc_entrenamiento[0, 0]), 'predicted_1': int(mc_entrenamiento[0, 1])},
    'true_1': {'predicted_0': int(mc_entrenamiento[1, 0]), 'predicted_1': int(mc_entrenamiento[1, 1])}
}

# Calcular la matriz de confusión para el conjunto de prueba
mc_prueba = confusion_matrix(y_prueba, y_prueba_pred)
mc_prueba_dict = {
    'type': 'cm_matrix',
    'dataset': 'test',
    'true_0': {'predicted_0': int(mc_prueba[0, 0]), 'predicted_1': int(mc_prueba[0, 1])},
    'true_1': {'predicted_0': int(mc_prueba[1, 0]), 'predicted_1': int(mc_prueba[1, 1])}
}

# Guardar las métricas y matrices de confusión en un archivo JSON
with open("../files/output/metrics.json", "w") as f:
    json.dump(metricas_entrenamiento, f)
    f.write('\n')  
    json.dump(metricas_prueba, f)
    f.write('\n')  
    json.dump(mc_entrenamiento_dict, f)
    f.write('\n')  
    json.dump(mc_prueba_dict, f)
    f.write('\n')
    
    
    